# Parallel Execution
Demonstrates how ray can be used to parallelize cross-validation.

In [1]:
import dask
import numpy as np
from time import sleep
from sklearn.model_selection import KFold
import copy

from dask.diagnostics import ProgressBar


In [2]:
from dask.distributed import Client

client = Client(n_workers=8,threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63734,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:63774,Total threads: 1
Dashboard: http://127.0.0.1:63775/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:63742,


In [7]:
def calc_duple(x):
    return (x, [ f'{x}', f'{x*2}' ])

In [11]:
futures = []
for i in range(3):
    futures += [ client.submit(calc_duple,i) ]

In [12]:
res = client.gather(futures)

In [15]:
for r in res:
    print(r[1])

['0', '0']
['1', '2']
['2', '4']


In [27]:
class TestClass:
    
    def __init__(self):
        return
        
    def worker(self,x):
        return (x, [ f'{x}', f'{x*2}' ])        
    
    
    def caller(self,client):
        
        futures = []
        for i in range(3):
            futures += [ client.submit(self.worker,i) ]    

        res = client.gather(futures)
        
        for r in res:
            print(r)
    
    

In [28]:
x = TestClass()
x.caller(client)

(0, ['0', '0'])
(1, ['1', '2'])
(2, ['2', '4'])


In [148]:
nFtr = 10
nEx = 10000000
np.random.seed(0)
covTbl = np.random.randint(0,2,size=(nEx,nFtr)).astype(np.uint8)
labels = np.random.randint(0,2,size=(nEx,)).astype(np.uint8)
rules = [ [ 2, 4 ], [ 0, 3, 5 ] ]
#head = np.full(nEx,False)
head = np.full(nEx,0).astype(np.uint8)

In [149]:
def my_func():

    head.fill(0) 
    for rule in rules:
#        _ = np.prod(covTbl[:,rule],axis=1)
#        head = np.logical_or(head,np.prod(covTbl[:,rule],axis=1))
        head += np.prod(covTbl[:,rule],axis=1,dtype=np.uint8)

    head = (head > 0).astype(np.uint8)

    tPos = np.sum(head * labels,dtype=int) 
    fPos = np.sum(head * (1-labels),dtype=int) 

    return tPos


In [150]:
%%time

tot = 0
for _ in range(10):
#    head.fill(False) 

    
    tot += my_func()
        
        
        

UnboundLocalError: local variable 'head' referenced before assignment

In [143]:
tPos

1720665

In [115]:
head.dtype

dtype('uint8')

In [165]:
import scipy as sci

In [169]:
nFtr = 250
depth = 3

sci.special.comb(250,3)*nFtr

643250000.0

In [170]:
from pympler.asizeof import asizeof

In [172]:
nEl = 1000000
asizeof(list(range(nEl)))

40000048

In [174]:
asizeof(np.arange(0,nEl,dtype=np.ushort))

2000120

In [175]:
x = np.array([12, 13, 14 ])
y = [ x ]
y

[array([12, 13, 14])]

In [117]:
print(client)

<Client: 'tcp://127.0.0.1:61310' processes=8 threads=8, memory=16.00 GiB>


In [6]:
@dask.delayed
def inc(x):
    return x + 1

@dask.delayed
def double(x):
    return x * 2

@dask.delayed
def add(x, y):
    return x + y

data = [1, 2, 3, 4, 5]

output = []
for x in data:
    a = inc(x)
    b = double(x)
    c = add(a, b)
    output.append(c)

total = dask.delayed(sum)(output)

In [7]:
total.compute()

50

In [8]:
nFeatures = 5
data = np.array([
    [ 1, 0, 1, 1, 0 ],
    [ 1, 0, 1, 0, 0 ],    
    [ 1, 0, 0, 1, 0 ],    
    [ 1, 0, 1, 1, 1 ],
    [ 1, 0, 1, 1, 1 ],
    [ 1, 0, 0, 0, 0 ],    
    [ 1, 0, 1, 1, 0 ],    
    [ 1, 0, 0, 1, 0 ],
    [ 1, 0, 0, 0, 0 ],
    [ 1, 1, 1, 1, 1 ] ])

labels = np.array([ 1, 0, 0, 0, 0, 0, 1, 0, 1, 1 ])

In [9]:
# Assume a simple majority vote type classfier.
def evaluate_fold(data,labels,trainIdx,testIdx):
    
    head = np.sum(data[testIdx,:],axis=1) > nFeatures/2
    tPos = np.sum(head * labels[testIdx])
    fPos = np.sum(head * (1-labels[testIdx]))
    nPos = np.sum(labels[testIdx])
    
    tmp = np.zeros((1,1000000))
    
    sleep(1)
    return np.array([tPos,fPos,nPos])

In [10]:
results = []

kf = KFold(n_splits=3)

for trainIdx, testIdx in kf.split(data,labels):            
    results += [ client.submit(evaluate_fold,data,labels,trainIdx,testIdx) ]
    


In [11]:
results

[<Future: pending, key: evaluate_fold-1a0d0f9f5d792d061db8201081de3b17>,
 <Future: pending, key: evaluate_fold-a7e4f4fc4d72151a2cb253d59fd987f9>,
 <Future: pending, key: evaluate_fold-bdff57e3d6b8320e9ddb040f296674cc>]

In [12]:
client.gather(results)

[array([1, 1, 1]), array([1, 1, 1]), array([1, 0, 2])]